In [3]:
!pip uninstall tensorflow

In [4]:
!pip install tensorflow==2.0.0-beta1

     |████████████████████████████████| 87.9MB 81kB/s 
     |████████████████████████████████| 501kB 40.4MB/s 
     |████████████████████████████████| 3.1MB 38.1MB/s 


## Part 1. Preparation 

In [2]:
import numpy as np
import random, multiprocessing
from datetime import datetime
import nltk
from nltk.corpus import brown
from gensim.models import Word2Vec
import tensorflow as tf
from collections import Counter
from itertools import chain
print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

2.0.0-beta1


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [3]:
nltk.download('brown')
nltk.download('universal_tagset')
nltk.download('stopwords')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Part 2. Data Preprocessing

### 2.1 Discover Details of the Dataset

In [0]:
brown.categories()

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

In [15]:
print(len(brown.words()))

1161192


In [0]:
print(len(brown.fileids()))
# There are only 500 documents. 
# So I have no idea why you set the limit of documentation size as 20000.

500


### 2.2 Hyper-parameter Preset

In [0]:
EMB_DIM = 300
w2v = Word2Vec(brown.sents(), size = EMB_DIM, window=5, min_count=5, negative=15,
               iter=10, workers=3)

In [5]:
word_vectors = w2v.wv # get trained embeddings
result = word_vectors.most_similar(positive = ['child'], negative = ['person'])
print("Most similar to 'child' but dissimilar to 'person': \n", result[:3])

Most similar to 'child' but dissimilar to 'person': 
 [('voice', 0.36667150259017944), ('smile', 0.3282575011253357), ('Pamela', 0.326852411031723)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
UNK_INDEX = 0
UNK_TOKEN = "UNK"

In [0]:
HIDDEN_SIZE = 50
BATCH_SIZE = 128
MINI_BATCH_SIZE = 10

### 2.3 Drop the common words

In [0]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))

In [0]:
source_list = brown.words()
counts = Counter(chain(*map(set, source_list)))

In [0]:
tags = [tag for (word, tag) in brown.tagged_words()]
print(len(set(tags)))

472


In [0]:
print(tags[:20])

['AT', 'NP-TL', 'NN-TL', 'JJ-TL', 'NN-TL', 'VBD', 'NR', 'AT', 'NN', 'IN', 'NP$', 'JJ', 'NN', 'NN', 'VBD', '``', 'AT', 'NN', "''", 'CS']


In [0]:
brown.tagged_words()[:30]

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.'),
 ('The', 'AT'),
 ('jury', 'NN'),
 ('further', 'RBR'),
 ('said', 'VBD'),
 ('in', 'IN')]

In [0]:
for i in source_list:
  print(i.ca)

In [0]:
brown.tagged_words(tagset='universal')

[('The', 'DET'), ('Fulton', 'NOUN'), ...]

In [0]:
print(len(brown.tagged_words(tagset='universal')))

1161192


In [0]:
print(len(source_list))

1161192


In [0]:
for genre in brown.categories():
  [i for i in brown.words(categories=genre) if counts[i]==1]
print()

### 2.4 Split Train and Test

In [0]:
brown_tagged_sents = brown.tagged_sents()

In [0]:
tag_words = brown.tagged_words()

In [9]:
print(tag_words[:10])

[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN')]


In [10]:
for item in tag_words[:10]:
  print(item[1])

AT
NP-TL
NN-TL
JJ-TL
NN-TL
VBD
NR
AT
NN
IN


In [15]:
bigram_tagger = nltk.bigrams(tag_words)
tags = [b[1] for (a,b) in  bigram_tagger]
fd = nltk.FreqDist(tags)
fd.tabulate()

         NN          IN          AT          JJ           .           ,         NNS          CC          RB          NP          VB         VBN         VBD          CS         PPS         VBG         PP$          TO        PPSS          CD       NN-TL          MD         PPO         BEZ        BEDZ          AP          DT          ``          ''          QL         VBZ          BE          RP         WDT         HVD           *         WRB         BER       JJ-TL       NP-TL          HV         WPS          --         BED         ABN         DTI          PN         NP$         BEN         DTS         HVZ           )           (      NNS-TL          EX         JJR          OD          NR           :         NN$       IN-TL       NN-HL          DO         NPS         PPL         RBR         DOD         JJT       CD-TL         MD*       AT-TL         ABX         BEG      NNS-HL          UH        .-HL      VBN-TL       NP-HL       IN-HL         DO*     PPSS+MD         DOZ       CD-HL     

In [0]:
num_labels = len(set(tags))

In [32]:
print(num_labels)

472


In [40]:
vocabulary_size = 2000
usage_tag_words = tag_words[:vocabulary_size]
size = int(vocabulary_size * 0.75)
print(size)
train_words = usage_tag_words[:size]
test_words = usage_tag_words[size:]

1500


In [0]:
for item in train_sents:
  print(item)

### 2.5 Build the pipeline

In [0]:
from tensorflow.keras.layers import Dense, Embedding, Activation, Flatten
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
import collections

In [0]:
def get_tag_vocabulary(tagged_words):
  tag2id = {}
  for item in tagged_words:
    tag = item[1]
    tag2id.setdefault(tag, len(tag2id))
  return tag2id

word2id = {k:v.index for k,v in word_vectors.vocab.items()}
tag2id = get_tag_vocabulary(train_words)

In [42]:
def get_int_data(tagged_words, word2id, tag2id):
  X, Y = [], [] # X for word ids, Y for tags
  unk_count = 0

  for word, tag in tagged_words:
    Y.append(tag2id.get(tag))
    if word in word2id:
      X.append(word2id.get(word))
    else:
      X.append(UNK_INDEX)
      unk_count += 1
  print("Data created. Percentage of unknown words: %.3f" % (unk_count/len(tagged_words)))
  return np.array(X), np.array(Y)

X_train, Y_train = get_int_data(train_words, word2id, tag2id)
X_test, Y_test = get_int_data(test_words, word2id, tag2id)

Data created. Percentage of unknown words: 0.047
Data created. Percentage of unknown words: 0.034


In [0]:
# One-hot encoding
Y_train, Y_test = to_categorical(Y_train), to_categorical(Y_test)

In [0]:
def add_new_word(new_word, new_vector, new_index, embedding_matrix, word2id):
  embedding_matrix = np.insert(embedding_matrix, [new_index], [new_vector], axis = 0)

  word2id = {word:(index + 1) if index >= new_index else index
             for word, index in word2id.items()}
  word2id[new_word] = new_index

  return embedding_matrix, word2id

embedding_matrix = word_vectors.vectors
unk_vector = embedding_matrix.mean(0)
embedding_matrix, word2id = add_new_word(UNK_TOKEN, unk_vector,
                                         UNK_INDEX, embedding_matrix, word2id)

## Part 3. Model

### 3.1 Define the Model

In [38]:
def init_model(embedding_matrix, class_count):
  vocab_length = len(embedding_matrix)
  weights = [embedding_matrix]
  bias = tf.Variable(tf.zeros([num_labels]))
  #logits = tf.matmul(X_train, weights) + bias
  #sce_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
  #    labels = Y_train, logits = logits))

  model = Sequential()

  model.add(Embedding(input_dim=vocab_length,
                      output_dim=EMB_DIM,
                      weights=weights,
                      input_length=1))
  model.add(Flatten())
  model.add(Dense(HIDDEN_SIZE))
  model.add(Activation("tanh"))
  model.add(Dense(class_count))
  model.add(Activation("softmax"))

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])
  return model

pos_model = init_model(embedding_matrix, len(tag2id))
pos_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 300)            4552200   
_________________________________________________________________
flatten_2 (Flatten)          (None, 300)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                15050     
_________________________________________________________________
activation_4 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 428)               21828     
_________________________________________________________________
activation_5 (Activation)    (None, 428)               0         
Total params: 4,589,078
Trainable params: 4,589,078
Non-trainable params: 0
____________________________________________

### 3.2 Train the Model 

In [43]:
pos_model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=50, verbose=1)

Train on 1500 samples
Epoch 1/50
1500/1500 [==============================] - 1s 697us/sample - loss: 6.4958 - accuracy: 0.0767
Epoch 2/50
1500/1500 [==============================] - 1s 618us/sample - loss: 4.5856 - accuracy: 0.1267
Epoch 3/50
1500/1500 [==============================] - 1s 654us/sample - loss: 3.0370 - accuracy: 0.2260
Epoch 4/50
1500/1500 [==============================] - 1s 626us/sample - loss: 2.2347 - accuracy: 0.4240
Epoch 5/50
1500/1500 [==============================] - 1s 621us/sample - loss: 1.7745 - accuracy: 0.5387
Epoch 6/50
1500/1500 [==============================] - 1s 607us/sample - loss: 1.4932 - accuracy: 0.5927
Epoch 7/50
1500/1500 [==============================] - 1s 607us/sample - loss: 1.2905 - accuracy: 0.6633
Epoch 8/50
1500/1500 [==============================] - 1s 611us/sample - loss: 1.1446 - accuracy: 0.7153
Epoch 9/50
1500/1500 [==============================] - 1s 598us/sample - loss: 1.0247 - accuracy: 0.7527
Epoch 10/50
1500/1500 [=

After 50 epochs of training, the accuracy has been improved from 7.67% to 92.8%